In [1]:
# OBJETIVO:
# Implementar o cálculo de métricas para aferir o quanto as representações vetoriais
# de documentos, produzidas por um modelo treinado, refletem suas similaridades.
#
# Previamente, um conjunto de N documentos de teste formado por pelo menos dois grupos
# de documentos sabidamente similares entre si será submetido ao modelo treinado,
# que em resposta fornecerá as representações vetoriais ("embeddings") desses documentos
# para que delas seja calculada a matriz de similaridade NxN.
#
# Serão então computadas métricas por dois meios:
# 1) média do CG (Cumulative Gain) e do nDCG (Normalized Discounted Cumulative Gain)
#    para as p melhores recomendações obtidas para cada documento representado na matriz
#    de similaridade, ignorado o próprio, com score binário, ou seja, 1=documento de
#    mesmo rótulo, 0=documento de rótulo diferente.
# 2) para o desempenho da clusterização da matriz de similidaridade dos documentos,
#    utilizando "spectral clustering". A premissa é que as representações vetoriais
#    serão tão melhores quanto maior for a correspondência entre os clusters obtidos
#    e os grupos de documentos sabidamente similares.

In [2]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.cluster import SpectralClustering

In [3]:
# função para calcular o CG (Cumulative Gain) na posição 'p' a partir de um array 1D
# de relevâncias (inclusive binárias) ordenado, onde o primeiro elemento é o score da
# primeira recomendação do modelo.
def calc_cg(relevancia, p):
    if p < len(relevancia):
        relevancia = relevancia[:p]
    return sum(relevancia)

In [4]:
# função para calcular o DCG na posição 'p' a partir de um array 1D de relevâncias (inclusive
# binárias) ordenado, onde o primeiro elemento é o score da primeira recomendação do modelo.
# utiliza a formulação que penaliza mais fortemente o score quando um documento de maior
# relevância é classificado abaixo de outro de menor relevância (equivale à formulação
# tradicional quando a relevância é binária).
# https://en.wikipedia.org/wiki/Discounted_cumulative_gain
def calc_dcg(relevancia, p):
    if p < len(relevancia):
        relevancia = relevancia[:p]
    num = 2**relevancia - 1
    den = np.log2(np.arange(len(relevancia)) + 2)
    return (num / den).sum()

In [5]:
# função para calcular o iDCG a partir do número de documentos relevantes esperado
# no resultado e do tamanho da lista de recomendações (p), considerando score
# binário (1=relevante, 0=não relevante).
def calc_idcg(num_relevantes, p):
    # array 1D com p zeros
    relevancia_ideal = np.zeros(p).astype(int)
    # atribui score=1 aos primeiros elementos, tantos quantos forem os relevantes
    n = min(num_relevantes, p)
    relevancia_ideal[:n] = 1
    return calc_dcg(relevancia_ideal, p)

In [6]:
# função para calcular o nDCG na posição 'p' a partir do iDCG e de um array 1D
# de relevâncias (inclusive binárias) ordenado, onde o primeiro elemento é o
# score da primeira recomendação do modelo
def calc_ndcg(relevancia, idcg, p):
    return calc_dcg(relevancia, p) / idcg

In [7]:
# função para calcular a média do CG (Cumulative Gain) e a média do nDCG (Normalized
# Discounted Cumulative Gain) para as 'p' melhores recomendações obtidas para cada
# documento representado na matriz de similaridade, ignorado o próprio, com score
# binário, ou seja, 1=documento de mesmo rótulo, 0=documento de rótulo diferente
def calc_ranking_metrics(similarity, labels_true, p):
    n = similarity.shape[0]
    if len(labels_true) != n:
        raise ValueError('Dimensao dos rotulos incompativel com a da matriz de similaridade')
    if p > (n - 1):
        raise ValueError('Ranking maior do que os possiveis resultados')
    sum_ndcg = 0
    sum_cg = 0
    for i in range(0, n):
        # rótulo de doc[i]
        label = labels_true[i]
        # número de documentos com mesmo rótulo de doc[i]
        num_same_label = 0
        # lista de tuplas de similaridade entre doc[i] e doc[j], e rótulo de doc[j]
        tuples = []
        for j in range(0, n):
            # ignora o próprio
            if i == j:
                continue
            if labels_true[j] == label:
                num_same_label += 1
            tuples.append((similarity[i, j], labels_true[j]))
        # calcula o idcg
        idcg = calc_idcg(num_same_label, p)
        # ordena em ordem decrescente de similaridade
        ranking = sorted(tuples, reverse=True, key=lambda t:t[0])
        # calcula as relevâncias binárias
        relevance = [1 if t[1] == label else 0 for t in ranking]
        # calcula o CG
        cg = calc_cg(np.array(relevance), p)
        sum_cg += cg
        # calcula o nDCG
        ndcg = calc_ndcg(np.array(relevance), idcg, p)
        sum_ndcg += ndcg
    return sum_cg / n, sum_ndcg / n

In [8]:
# função para a clusterização da matriz de similaridade NxN obtida das
# representações vetoriais NxV de N documentos, e para o cálculo de métricas
# de desempenho, a partir da lista de rótulos conhecidos de cada documento
def calc_cluster_metrics(similarity, labels_true):
    if len(labels_true) != similarity.shape[0]:
        raise ValueError('Dimensao dos rotulos incompativel com a da matriz de similaridade')
    # o número de clusters é o número de rótulos distintos
    clusters = len(set(labels_true))
    # clusterização com "spectral clustering"
    sc = SpectralClustering(n_clusters=clusters, assign_labels="discretize", affinity='precomputed')
    labels_pred = sc.fit_predict(similarity)
    print('Clusters:', clusters)
    print('Clustering (ground-truth):')
    print(np.array(labels_true))
    print('Clustering (predicted):')
    print(labels_pred)
    print('Homogeneity: %0.3f' % metrics.homogeneity_score(labels_true, labels_pred))
    print('Completeness: %0.3f' % metrics.completeness_score(labels_true, labels_pred))
    print('V-measure: %0.3f' % metrics.v_measure_score(labels_true, labels_pred))
    print('Adjusted Rand Index (ARI): %0.3f' % metrics.adjusted_rand_score(labels_true, labels_pred))
    print('Adjusted Mutual Information (AMI): %0.3f' 
          % metrics.adjusted_mutual_info_score(labels_true, labels_pred))
    print('Fowlkes-Mallows Index (FMI): %0.3f'
          % metrics.fowlkes_mallows_score(labels_true, labels_pred))
    return labels_pred

In [9]:
# Exemplo - modelo ideal:
similarity = np.array([[1,1,0,0,0,0],[1,1,0,0,0,0],[0,0,1,1,0,0],[0,0,1,1,0,0],[0,0,0,0,1,1],[0,0,0,0,1,1]])
cols = ['96-A','96-B','113-A','113-B','120-A','120-B']
pd.DataFrame(data=similarity, index=cols, columns=cols)

,96-A,96-B,113-A,113-B,120-A,120-B
96-A,1,1,0,0,0,0
96-B,1,1,0,0,0,0
113-A,0,0,1,1,0,0
113-B,0,0,1,1,0,0
120-A,0,0,0,0,1,1
120-B,0,0,0,0,1,1


In [10]:
# Exemplo - modelo ideal (continuação):
labels_true = np.array([96, 96, 113, 113, 120, 120])
mean_cg, mean_ndcg = calc_ranking_metrics(similarity, labels_true, 3)
print('CG médio: %0.3f' % mean_cg)
print('nDCG médio: %0.3f' % mean_ndcg)
calc_cluster_metrics(similarity, labels_true)

CG médio: 1.000
nDCG médio: 1.000
Clusters: 3
Clustering (ground-truth):
[ 96  96 113 113 120 120]
Clustering (predicted):
[2 2 0 0 1 1]
Homogeneity: 1.000
Completeness: 1.000
V-measure: 1.000
Adjusted Rand Index (ARI): 1.000
Adjusted Mutual Information (AMI): 1.000
Fowlkes-Mallows Index (FMI): 1.000


C:\Users\76360784653\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


array([2, 2, 0, 0, 1, 1], dtype=int64)

In [11]:
# Exemplo - modelo real:
similarity = np.array([[1.0, 0.3, 0.7, 0.6, 0.1, 0.1],
                       [0.3, 1.0, 0.1, 0.1, 0.1, 0.1],
                       [0.7, 0.1, 1.0, 1.0, 0.1, 0.1],
                       [0.6, 0.1, 1.0, 1.0, 0.1, 0.1],
                       [0.1, 0.1, 0.1, 0.1, 1.0, 1.0],
                       [0.1, 0.1, 0.1, 0.1 ,1.0, 1.0]])
pd.DataFrame(data=similarity, index=cols, columns=cols)

,96-A,96-B,113-A,113-B,120-A,120-B
96-A,1.0,0.3,0.7,0.6,0.1,0.1
96-B,0.3,1.0,0.1,0.1,0.1,0.1
113-A,0.7,0.1,1.0,1.0,0.1,0.1
113-B,0.6,0.1,1.0,1.0,0.1,0.1
120-A,0.1,0.1,0.1,0.1,1.0,1.0
120-B,0.1,0.1,0.1,0.1,1.0,1.0


In [12]:
mean_cg, mean_ndcg = calc_ranking_metrics(similarity, labels_true, 3)
print('CG médio: %0.3f' % mean_cg)
print('nDCG médio: %0.3f' % mean_ndcg)
calc_cluster_metrics(similarity, labels_true)

CG médio: 1.000
nDCG médio: 0.917
Clusters: 3
Clustering (ground-truth):
[ 96  96 113 113 120 120]
Clustering (predicted):
[1 0 1 1 2 2]
Homogeneity: 0.710
Completeness: 0.772
V-measure: 0.740
Adjusted Rand Index (ARI): 0.444
Adjusted Mutual Information (AMI): 0.502
Fowlkes-Mallows Index (FMI): 0.577


array([1, 0, 1, 1, 2, 2], dtype=int64)